In [7]:
import esm
import pyvista as pv
import os
import pandas as pd
import numpy as np
import trimesh as tm
import itertools

cwd = os.getcwd()
src = os.path.dirname(cwd)

In [8]:
# import all external geometry with pyvista
wal_filepath = os.path.join(src, "data", "test_room_walls_02.obj")
wal_mesh = pv.read(wal_filepath)
fen_filepath = os.path.join(src, "data", "test_room_fens_02.obj")
fen_mesh = pv.read(fen_filepath)

con_buildings_filepath = os.path.join(src, "data", "test_context_01_buildings.obj")
con_buildings_mesh = pv.read(con_buildings_filepath)
con_ground_filepath = os.path.join(src, "data", "test_context_01_ground.obj")
con_ground_mesh = pv.read(con_ground_filepath)
# con_sky_filepath = os.path.join(src, "data", "test_context_01_sky.obj")
# con_sky_mesh = pv.read(con_sky_filepath)
con_vegetation_filepath = os.path.join(src, "data", "test_context_01_vegetation.obj")
con_vegetation_mesh = pv.read(con_vegetation_filepath)

# construct honeybee surfaces from the mesh
wal_hb_surf = esm.pv_mesh_to_hbsurface(wal_mesh, 0, "wall", esm.material_plastic)
fen_hb_surf = esm.pv_mesh_to_hbsurface(fen_mesh, 5, "window", esm.material_glass)
con_buildings_hb_surf = esm.pv_mesh_to_hbsurface(con_buildings_mesh, 0, "context_buildings", esm.material_plastic)
con_ground_hb_surf = esm.pv_mesh_to_hbsurface(con_ground_mesh, 0, "context_ground", esm.material_plastic)
# con_sky_hb_surf = esm.pv_mesh_to_hbsurface(con_sky_mesh, 0, "context_sky", esm.material_plastic)
con_vegetation_hb_surf = esm.pv_mesh_to_hbsurface(con_vegetation_mesh, 0, "context_vegetation", esm.material_plastic)

hb_surfaces = wal_hb_surf + fen_hb_surf + con_buildings_hb_surf + con_ground_hb_surf + con_vegetation_hb_surf

In [19]:
# homogenous discretization of sphere
icosphere = tm.creation.icosphere(subdivisions=5, radius=1.0)
v = icosphere.vertices

# setting the rays and points
rays = [tuple(v) for v in icosphere.vertices]
pnts = [(1, 1, 1),(6, 1, 1),(1, 6, 1),(1, 1, 6)]

# generate grid of test points
test_inds = [j for j in range(len(pnts)) for i in range(len(rays))]
test_pnts = [pnt for pnt in pnts for i in range(len(rays))]
test_rays = rays * len(pnts)

analysis_grid = esm.AnalysisGrid.from_points_and_vectors(test_pnts, test_rays, 'test_grid')

# put the recipe together
rp = esm.ContextViewGridBased(analysis_grids=(analysis_grid,),
               hb_objects=hb_surfaces)

Found 5486 opaque surfaces.
Found 1 fenestration surfaces.
Found 0 window-groups.


In [10]:
# write simulation to folder
project_name = 'clean_context_view'
batch_file = rp.write(
    target_folder='.', project_name=project_name)


Writing recipe contents to: ./clean_context_view/gridbased


In [11]:
# run the simulation
rp.run(batch_file, debug=False)

True

In [26]:
# load rtrace results
rs_path = os.path.join(project_name, 'gridbased', 'rtrace_res.txt')
rtrace_res = pd.read_csv(rs_path, skiprows=8, sep='\t', usecols=[3,4,5], header=None, names=['last_int_name', 'first_int_dist', 'last_int_dist'])

# load the rays
rays_path = os.path.join(project_name, 'gridbased', 'clean_context_view.pts')
rays_df = pd.read_csv(rays_path, skiprows=0, sep=' ', header=None, names=['Sx','Sy','Sz','Rx', 'Ry', 'Rz'])

# concatenate data frames and group by point ids
grid_results = pd.concat([rays_df, rtrace_res], axis = 1)
gb = grid_results.groupby(test_inds)

# extract point 0 info
point_0_results = gb.get_group(0)
point_0_results

,Sx,Sy,Sz,Rx,Ry,Rz,last_int_name,first_int_dist,last_int_dist
0,1.0,1.0,1.0,-0.526,0.851,0.000,wall,1.901974,1.901974
1,1.0,1.0,1.0,0.526,0.851,0.000,context_buildings,4.205255,4.205255
2,1.0,1.0,1.0,-0.526,-0.851,0.000,wall,1.175603,1.175603
3,1.0,1.0,1.0,0.526,-0.851,0.000,window,119.185700,1.175603
4,1.0,1.0,1.0,0.000,-0.526,0.851,wall,1.901974,1.901974
...,...,...,...,...,...,...,...,...,...
10237,1.0,1.0,1.0,-0.954,-0.301,0.021,wall,1.048825,1.048825
10238,1.0,1.0,1.0,-0.939,-0.334,0.082,wall,1.064963,1.064963
10239,1.0,1.0,1.0,-0.930,-0.368,0.021,wall,1.075680,1.075680
10240,1.0,1.0,1.0,-0.952,-0.300,0.061,wall,1.050433,1.050433


In [27]:
window_mask = (point_0_results['last_int_dist'] - point_0_results['first_int_dist']).abs() > 0
print('window view solid angle:',4 * np.pi * window_mask.sum()/len(window_mask), 'steradian')

window view solid angle: 1.9459347582868236 steradian


In [28]:
horizontal_line_mask = point_0_results['Rz'].abs() < 0.01
print('horizontal window view angle:',(2 * np.pi * window_mask * horizontal_line_mask).sum() / horizontal_line_mask.sum(), 'radians')

horizontal window view angle: 2.012582793705961 radians


In [29]:
view_quality_ratio = point_0_results['last_int_name'].value_counts() / len(point_0_results)
view_quality_ratio

wall                 0.553408
context_ground       0.257860
window               0.154853
context_buildings    0.033880
Name: last_int_name, dtype: float64

In [31]:
p = pv.Plotter(notebook=True)

icosphere = tm.creation.icosphere(subdivisions=5, radius=1.0)

p.add_mesh(icosphere.vertices, scalars=point_0_results['last_int_dist'], point_size=10)

p.add_axes()

p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(3.8637033051562737, 3.8637033051562737, 3.8637033051562737),
 (0.0, 0.0, 0.0),
 (0.0, 0.0, 1.0)]